In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ratings_data = {
    "User": ["Babs_05", "Orlenay", "massdosage", "Knapster01", "eartle"],
    "Samsung AC (1.5 Ton)": [4.1, 5.0, 4.0, 5.0, 3.0],
    "Voltas AC (1.5 Ton)": [np.nan, 2.6, 3.5, 2.5, 2.0],
    "Samsung AC (1 Ton)": [4.0, 3.0, 2.4, 3.4, 4.0],
    "LG AC (1 Ton)": [4.2, 4.3, 3.4, 2.1, 2.6]
}

data = pd.DataFrame(ratings_data).set_index("User")

def compute_cosine_similarity(matrix):
    sim_matrix = cosine_similarity(matrix.fillna(0))
    return pd.DataFrame(sim_matrix, index=matrix.index, columns=matrix.index)

def compute_pearson_similarity(matrix):
    return matrix.transpose().corr(method="pearson")

def predict_ratings(user_item_matrix, similarity_matrix, user_based=True):
    predictions = user_item_matrix.copy()
    for user in user_item_matrix.index:
        for item in user_item_matrix.columns:
            if pd.isna(user_item_matrix.loc[user, item]):
                if user_based:
                    sim_scores = similarity_matrix[user]
                    item_ratings = user_item_matrix[item]
                else:
                    sim_scores = similarity_matrix[item]
                    item_ratings = user_item_matrix.loc[user]

                valid_ratings = item_ratings[~item_ratings.isna()]
                valid_sim_scores = sim_scores[valid_ratings.index]

                if valid_sim_scores.sum() != 0:
                    predicted_value = (valid_sim_scores * valid_ratings).sum() / valid_sim_scores.sum()
                else:
                    predicted_value = np.nan

                predictions.loc[user, item] = predicted_value
    return predictions